In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

%matplotlib inline
np.random.seed(1)

In [0]:
plt.rcParams['figure.figsize'] = (8.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

np.random.seed(1)

## 1 - Overview of the data set

First of all, we'll load the data. After loading, here's a basic overview;

- a training set of m_train images labeled as cat (y=1) or non-cat (y=0)
- a test set of m_test images labeled as cat or non-cat
- each image is of shape (num_px, num_px, 3) where 3 is for the 3 channels (RGB). Thus, each image is square (height = num_px) and (width = num_px).

In [0]:
train_dataset = h5py.File('./train.h5', "r")
train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # train set features
train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # train set labels

test_dataset = h5py.File('./test.h5', "r")
test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # test set features
test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # test set labels

In [0]:
classes = np.array(test_dataset["list_classes"][:]) # the list of classes
classes

In [0]:
train_set_y = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
test_set_y = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
print(train_set_y.shape)
print(test_set_y.shape)

Each line of train_set_x_orig and test_set_x_orig is an array representing an image. You can visualize an example by running the following code. Feel free also to change the index value and re-run to see other images.

In [0]:
# Example of a picture
index = 60
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]) + ", it's a '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "' picture.")

# 2 - Data preprocessing

Following are the steps to preprocess data for a ConvNet.

- Figure out the dimensions and shapes of the problem (m_train, m_test, num_px, ...)
- "Standardize" the data
- Data Augmentation (will discuss later in the course)

Many software bugs in deep learning come from having matrix/vector dimensions that don't fit. If you can keep your matrix/vector dimensions straight you will go a long way toward eliminating many bugs.

### 2.1 Dimensions of our data

In [0]:
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

print ("Dataset dimensions:")
print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")

In [0]:
train_set_y = train_set_y.T
test_set_y = test_set_y.T

In [0]:
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

### 2.3 Standardizing data

One common preprocessing step in machine learning is to center and standardize your dataset, meaning that you substract the mean of the whole numpy array from each example, and then divide each example by the standard deviation of the whole numpy array. But for picture datasets, it is simpler and more convenient and works almost as well to just divide every row of the dataset by 255 (the maximum value of a pixel channel).

Let's standardize our dataset.

In [0]:
train_set_x = train_set_x_orig/255.
test_set_x = test_set_x_orig/255.

In [0]:
train_set_x.shape

## 3 - Modeling

### 3.1 Utility function to train & evaluate our models.

In [0]:
# Utility function
def evaluate_this_model(model, epochs):
    
    np.random.seed(1)

    history = model.fit(train_set_x, train_set_y, epochs=epochs)
    results = model.evaluate(test_set_x, test_set_y)
    
    plt.plot(np.squeeze(history.history["loss"]))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    print("\n\nAccuracy on training set is {}".format(history.history["acc"][-1]))
    print("\nAccuracy on test set is {}".format(results[1]))

### 3.2 Model Architecture

In [0]:
model = Sequential()

model.add(Conv2D(10, kernel_size=3, activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(15, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(20, padding="same", kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=35, activation='relu'))
model.add(Dense(units=15, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

### 3.3 Compilation

In [0]:
learning_rate = 0.0001
opt = Adam(lr=learning_rate)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
evaluate_this_model(model, 10)

## 4 - Improving Accuracy

In [0]:
def evaluate_my_model(epochs):
    
    np.random.seed(1)
    
    train_accuracies = []
    test_accuracies = []
    
    for i in range(5):
        model = get_model()
        history = model.fit(train_set_x, train_set_y, epochs=epochs)
        results = model.evaluate(test_set_x, test_set_y)
        train_accuracies.append(history.history["acc"][-1])
        test_accuracies.append(results[1])

    plt.plot(np.squeeze(history.history["loss"]))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Last iteration")
    plt.show()
    
    print(train_accuracies)
    print(test_accuracies)
    print("\n\nAccuracy on training set is {}".format(np.mean(train_accuracies)))
    print("\nAccuracy on test set is {}".format(np.mean(test_accuracies)))

In [0]:
def get_model():
    model = Sequential()
    
    model.add(Dense(units=1, activation='sigmoid'))
    
    learning_rate = 0.0001
    opt = Adam(lr=learning_rate)
    model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model

In [0]:
epochs = 5
evaluate_my_model(epochs)